# I'm Sorry Dave, I Can't Do That...
## Adventures in Machine Learning!
##### Copyright &copy; 2019, 2020 David Hoelzer / Enclave Forensics Inc., All Rights Reserved
> License is granted to copy, use, and reproduce this Notebook and Python code as-is for any non-commercial use.  This license may not be removed.  Any reuse of any portions of the code or text in this Notebook must be attributed with HTTP pointers to the original code.

This is a 60-90 minute crash-course excerpt from the new Applied Machine Learning for Information Security course that is in development.  We expect this to be released to the public in the May - June 2020 timeframe.

### What We Teach:
 * What is machine learning and AI today, really (We'll do this tonight *very* quickly
 * What limitations does ML and AI have today (We'll do this tonight *very, very* quickly
 * What useful things can I build **now**?  (We'll do this too!!!)
 
 ---

## What is Machine Learning?
### Linear Regression

* Take some data points
* Work out a function that approximates the data
* Hopefully, predict what that data will do in the future

There's just one thing we have to make sure you know to provide some context...

> # What is this the formula for?
> ## y = mx + b



In [ ]:
# Some quick Python to read in some sample data showing bytes transferred per hour
import numpy as np
import pandas as p
import os
import scipy as sp
from scipy.stats import gamma
import matplotlib.pyplot as plt

# Load Netflow statistics for multiple days
data = p.read_csv("flowStats.csv")

# Look at it
data

# Cleaning and Processing the Data
## Extra Column
The extra column is an artifact of the generating tool.  Let's recreate the dataframe, dropping the third column:

In [ ]:
data.drop(data.columns[2], axis=1, inplace=True)
data

## Training Data vs Verification Data
In order to know how well our approximation predicts the future, we need to hold back some data so that we can tell how well our result approximates reality or *Ground Truth* using known data that the system hasn't been trained on or fitted to.

`bytes` will hold the data we use for fitting or learning.

`allbytes` will hold the *ground truth* of all of the known data that we have.

In [ ]:
bytes = data.to_numpy()[0:60,1]
allbytes = data.to_numpy()[:,1]
# Print the result out so we can see if we got only byte counts.  The result will be in scientific notation by default.
bytes

# Hand-wavy Python Magic (Wibbly-Wobbly, Timey-Wimey Stuff)
What follows is some Python magic that:

* Gets rid of the nasty looking Epoch timestamps, trading them in for hour numbers starting from zero.
* Creates a handy function that will graph data for us.
* Includes some special Jupyter "magic" to display graphs inline in the notebook.

In [ ]:
# Create a set of hours that we can map this against.  Honestly,
# we could have simply used the entire data set as-is, but
# then we wouldn't have had an excuse to slice an array and
# create a range
hours = range(0,bytes.size)

def plot_data(x, y, models=None, mx=None, ymax=None, fig_idx=None):
    '''
    Plot the data (y) over time (x). 
    
    models, if present, is an array of polynomial models generated
    by polyfit()
    '''
    x = range(0,x[-1])
    plt.figure(figsize=(12,6), dpi=300) # width and height of the plot in inches
    plt.scatter(x, y, s=10)
    plt.title("Bytes per Hour")
    plt.xlabel("Time")
    plt.ylabel("Bytes")
    
    if models:
        # I'm only supporting six plots because that's all that I need. :)
        colors = ['g', 'k', 'b', 'm', 'r', 'i']
        linestyles = ['-', '-.', '--', '-', '-', '.']

        if mx is None:
            mx = np.linspace(0, len(x), num=20)
        for model, style, color in zip(models, linestyles, colors):
            plt.plot(mx, model(mx), linestyle=style, linewidth=2, c=color)

        plt.legend(["d=%i" % m.order for m in models], loc="upper left")
        
    plt.autoscale()
    if ymax:
        plt.ylim(ymax=ymax)

    plt.grid()
    plt.ylim(bottom=0)

# This next line is INCREDIBLY important!  When using iPython and Jupyter,
# if you want to get it to plot things inline as you are working with the
# notebook, you MUST include this line.  Without it, you will never see
# a graph!
%matplotlib inline

# Let's have a look at the data that we will use for training.
plot_data(range(0,len(bytes)+1), bytes)

## What Now?  Regression!

* Regression = "Fitting" or "Approximation"

In other words, can we come up with some function *f* that will produce something really close to those dots?

> *(Where **x** is the hour number and **y** is the number of bytes seen)*
> Said another more mathy way, can we find *f(x)* where *f(x)* accurately produces a result that matches ground truth?  Moving to this *f(x)* notation, *f(x)* is just a stand-in (or another way of saying) *y*.

> Since we are starting with the idea that *y = mx + b*, or *f(x) = mx + b*, we are attempting to find values for *m* and *b* that allow our line to closely match, approximate, or fit reality.

### Linear Regression = Draw A Line

In [ ]:
# We need an error function that gives us the average or mean of the squares 
# of the distances between the actual value and the model.
def mean_squared_error(f, x, y):
    return np.average((f(x)-y) **2)

In [ ]:
# Next, we try a naive approach of a straight line approximation of our function.
# The polyfit function performs this... regression analysis?  We can specify the order of the polynomial, which
# is 1 for a line:

fp1 = np.polyfit(range(0,len(hours)), bytes, 1)
print(f'After fitting for a first order equation, the result is {fp1}')
print(f'This translates to:   f(x) = {fp1[0]} * x + ({fp1[1]})')


In [ ]:
# These means that f(x) = 894781.28905251 * x + -10550937.97704918..., or y=mx + b
# To convert that array into a function, we pass it to np.poly1d, where 1d is the order of the function:
f1 = np.poly1d(fp1)
mean_squared_error(f1, hours, bytes)


In [ ]:
# Plotting the same scatter plot and passing f(x)=f1 - Note the square brackets.
plot_data(range(0,len(bytes)+1), bytes, [f1])


## Infinite Series

It is (generally) possible for any continuous curve to find a function that will "fit" that curve well.  For simple curves this is (relatively) simple.  For more complex curves this is more difficult...  For instance:


In [ ]:
sine_range = 10
range_to_plot = np.arange(-1 * sine_range,sine_range,0.1)
plt.figure(figsize=(12,6), dpi=300)
plt.plot(range_to_plot, (np.cos(range_to_plot)))
plt.title(f'A Plot of Sine from {-1 * sine_range} pi to {sine_range} pi')

## The More Terms...

We don't want to become too mathy here, but a Taylor series is an infinite series that can be used to approximate the value of Sine or Cosine (and other interesting values) *very* accurately.  *Why and how* this works is very interesting, and I encourage you to investigate the topic if you are curious.  It will give you a much better appreciation of how derivatives can be chained together to control a desired output... but that's not why we're here now.

> ### Infinite doesn't have to mean infinity!
> It turns out that you can be *very* accurate around a specific point where you need to approximate Sine using very few terms.  The further you venture from that point, however, the worse the approximation becomes...  Said another way, the further you wander away from ground truth...  Or the worse your error is... Which is also called **Loss**.

### Let's Try That (in a simpler way)

Fundamentally, the more terms we add, the better the approximation.  Adding a term is adding some exponential value of *x*.  Said another way, we are creating a *higher order equation*.

In [ ]:
import math

def taylor_cosine(x, n=1):
    """Returns the taylor series approximation of the cosine of X over N terms."""
#           oo              2n + 1
#          ---      n      x
#   sin x=  >   (-1)  ------------
#          ---         (2n + 1)!
#          n=0

    approximation = 0
    # In Python 3.8+ the following *should* work:
    # [approximation := approximation + x for x in ((-1)**i) * ((x**((2*i)+1))/(math.factorial(2*i))) for i in range(n)]
    # 
    # For now, we use a loop:
    for i in range(n):
        approximation += ((-1)**i) * ((x**((2*i)+1))/(math.factorial(2*i)))
    
    return approximation

taylor_terms_to_calculate = 13

plt.figure(figsize=(12,10), dpi=300)
plt.plot(range_to_plot, (np.cos(range_to_plot)))
plt.ylim(top=4,bottom=-4)
for i in range(1,taylor_terms_to_calculate):
    plt.plot(range_to_plot, [func_cos(x, i) for x in range_to_plot])

plot_legend = ['Cosine function'] + [(f'Taylor Terms: {x}') for x in range(1,taylor_terms_to_calculate) ]

plt.legend(plot_legend)
plt.title(f'Sine(x) Compared to Taylor Approximations Using Between 1 and {taylor_terms_to_calculate - 1} terms')

### Wow!
Have a look at the graph.  I know it's a lot to take in, but consider what's going on.  Find the Cosine graph and follow it.  Now find the horizontal line that is orange.  That horizontal line is the very first result of a Taylor series with one term.  At *x = 0* it exactly matches the value of *sin(0)*, but after that it only matches the value of *sin(x)* every *2$\pi$* (or 360$^\circ$) values !  The dark green graph, a Taylor series that computes two terms looks like it is "good enough" between perhaps *-0.5* and *0.5* radians.  If you scan down to the result of using six terms, the values very closely approximate *sin(x)* for a much wider range, nearly *-5.0* to *5.0*.  There are some diminishing returns, however.  The early success at very close approximation seems to decline rapidly.  Doubling the number of terms to 12 we find that the approximation is good for less than twice the range.

Even so, this means that we can calculate any arbitrary number of terms we need in order to approximate the value of Sine to any precision.

## Wait... What?
So what does this have to do with our topic?  Well, so far we've asked Python to come up with a first order equation to approximate our data.  Extending what we see with the Taylor series, it intuitively seems likely that providing a larger number of terms will allow us to fit the data more precisely!  Let's give that a try.

In [ ]:
# Clearly this is insufficient.  We can quickly try to fit to other functions by using polyfit to find 
# coefficients for higher order polynomials:

num_hours = range(0,len(bytes))

fp2 = sp.polyfit(num_hours, bytes, 2)
f2 = sp.poly1d(fp2)
fp3 = sp.polyfit(num_hours,bytes, 3)
f3 = sp.poly1d(fp3)
fp4 = sp.polyfit(num_hours,bytes, 15)
f4 = sp.poly1d(fp4)


# And let's look at the errors:
print(" 1st Order: ", error(f1, num_hours, bytes))
print(" 2nd Order: ",error(f2, num_hours, bytes))
print(" 4rd Order: ",error(f3, num_hours, bytes))
print("15th Order: ",error(f4, num_hours, bytes))
print("\nClearly the error is getting smaller, so the approximation must be better... Right??")

In [ ]:
num_hours = range(0,len(bytes)+1)
plot_data(num_hours, bytes, [f1,f2], mx = np.linspace(num_hours[0], num_hours[-1], 100))
plot_data(num_hours, bytes, [f1,f2, f3], mx = np.linspace(num_hours[0], num_hours[-1], 100))
plot_data(num_hours, bytes, [f1,f2, f3, f4], mx = np.linspace(num_hours[0], num_hours[-1], 100))



### What Does This Mean?
As you can see, the graphs start out looking promising.  The second order equation is better and the third order is even better!  If those are good, a 15th order equation must be amazing!!  But something has seemingly gone awry.

Notice that the 15th order equation starts out well, but it is no longer *generalizing* on the data.  Instead, it's almost as though it is darting from point to point, no longer able to "see" any overarching patterns in the data.  This is an artifact of overtraining and/or overfitting.  What does this mean?  If we overtrain or overfit our data, we are no longer generalizing.  Instead, our function is *very* good at finding points that are in the known training set, but likely very *bad* at finding or predicting anything else.  We can prove this by graphing out these functions with the hold-out data that we set aside (and we'll do that in a second).

Before moving on, recognize that generating a linear regression isn't actually the point here.  There are two much bigger points (or intuitions) that I'd like you to come away with:

* Machine Learning (which is how the linear regression was performed) is using an algorithm to adjust values ( coefficients in this case) to minimize the error (or loss) between the prediction (output of the function) and ground-truth (reality)
* Overtraining or overfitting will usually result in a model that performs exceptionally well on the *training data* but very poorly on everything else.

In [ ]:
num_hours = range(0,len(allbytes)+1)
plot_data(num_hours, allbytes, [f1,f2], mx = np.linspace(-10, num_hours[-1]+2, 100))
plot_data(num_hours, allbytes, [f1,f2, f3], mx = np.linspace(-10, num_hours[-1]+2, 100))
plot_data(num_hours, allbytes, [f1,f2, f3, f4], mx = np.linspace(-10, num_hours[-1]+30, 100))

It's quite clear that the overall predictions using regression fitting are pretty poor in this case.  Still, we've gained some valuable insights.

---

# The Story So Far:

* Machine Learning = Automatically adjusting variables to make a prediction
* The adjustments are made by minimizing the "Losses"
* Loss is the same as error
* Both of these are how far away from "Ground Truth" or reality the predicted value is
* Overfitting/overtraining happens when your approximation looks far too closely at the specifics, failing to generalize
    * This can also be us doing a bad job of feature selection, which more frequently how the term "overfitting" is used.


---


# Hello, world!
### A Quick Detour into Binary Classification

> Classifying movie reviews into two categories (binary classification) was a cutting edge problem in natural language and machine learning less than a decade ago.  While we can view this as a "solved" problem and there are absolutely better approaches than what we will do here, there are still some very valuable intuitions that we can learn by using this is a bridging example between simple linear regression and more complex logistic regressions

In [ ]:
import numpy as np

# We only need a few specific things from Keras, so we import only those elements.
from keras import models
from keras import layers

# Keras.datasets contains a number of interesting/useful datasets, especially when exploring and
# experimenting with machine learning.
from keras.datasets import imdb


## The Data
The IMDB review data is stored as two tuples of tuples.  These are a first tuple representing a set of 25,000 movie reviews (`train_data` in our experiment) and 25,000 classifications (`train_labels` in our experiment).  The second tuple contains a set of another 25,000 movie reviews (`test_data`) along with the matching classifications (`test_labels`).

The movie reviews were originally sentences made up of words, but these have been preprocessed to save some time.  The words have been replaced with numbers that represent an index into the set of words used in all reviews.  For example, imagine that we had the following index:

1. The
2. fox
3. brown
4. is
5. quick
6. lazy
7. jumps
8. dog
9. lazy
10. jumped
11. over

Given that index, the following sentence:

```
the quick brown fox jumps over the lazy dog
```

would be represented in our dataset as:

```
1 5 3 2 7 11 1 6 8
```

The table of words is ordered by frequency, meaning that word with index 1 occurs more frequently than any other word in the dataset.  If a word has an index of 0, this simply means that the word was unique in the data and was not stored.  Having the words organized in this way allows us to limit how many "features" (words) our network will have to work with.

# Python Magic Time!
Let's start by loading the data.

In [ ]:
# Let's define a variable representing the maximum number of words or features that we want
# to work with.  This is passed as a parameter to the "load_data" function for the imdb dataset
# from Keras.

WordsToWorkWith = 10000
(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=WordsToWorkWith)

# Eventually, we will likely want to look at some of the reviews ourselves, so let's load the word
# index from the dataset.  This is available with the function call get_word_index()
# get_word_index retrieves the table of words
word_index = imdb.get_word_index()

# Let's have a look at a little bit of the data so far:

print(f"train_data is of shape {train_data.shape} and train_labels is of shape {train_labels.shape}")
print(f"The first element in train_data is:\n\n{train_data[0]}\n\n")
print(f"The first train_label label is: {train_labels[0]}")
firstTenWords = {k: word_index[k] for k in list(word_index)[:10]}
print(f"The word index is a dictionary with {len(word_index.keys())} entries.  Here are the ten key-value pairs in it:\n{firstTenWords}")
      


## Making It Readable
Right now, the reviews are represented in a way that isn't particularly readable for us, nor is it structured in an ideal way for the path our experiment will take.  In a sense, the data is currently in a sort of in-between form.

It would be useful for us to be able to take the series of numbers in the data and turn that back into a set of words that we can read.  Doing so isn't actually difficult, but it will require some quick Python gymnastics because the `word_index` is in a dictionary where the word is the key and the index number is the value.  Why is it stored this way?  Because it makes it very easy to take new reviews and automatically convert them into this same format.  This would allow us to take our trained network and very quickly process new reviews to determine if they are negative reviews (a label value of 0) or positive reviews (a label value of 1).

The first review appears to be a positive review, but let's see for ourselves.

In [ ]:
# first, we will use some dictionary comprehension features of Python to reverse the keys and values
# in the word_index dictionary.  YOU DO NOT NEED TO BE ABLE TO DO THIS!  Remember, this isn't a Python
# class. :) . Googling for a "recipe" to reverse values in this way is the perfect way to get started.
# We've already figured this part out:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

# Next, we'll create a function that iterates over the values in a sequence of word indices and returns
# the matching words in that order.
def decodeReview(data, x):
    return ' '.join([reverse_word_index.get(i-3, '?') for i in data[x]])

# Let's also create a function that returns "Postive" or "Negative" based on the passed in label
def goodOrBad(x):
    return 'positive' if x==1 else 'negative'

# Now that we've got a function to decode a review, let's look at that review again:
review = decodeReview(train_data,0)
print(f"The first review states:\n\n{review}")
print(f"\n\nThis was a *{goodOrBad(train_labels[0])}*")


# Unpleasant Math Interlude

Much of machine learning today relies heavily on Linear Algebra.  Don't worry, we're not going to teach you this!  Linear Algebra, put very simply, the algebra of vectors.

* A single number is a "Scalar."  We'll explain why this is in a second.
* A vector is an ordered set of numbers.  For example *(x, y)* where *x=5* and *(y=1)* would define the coordinate (5,1)
    * Yes, I know you physics folks were taught that it has a position and a magnitude, but just go with this math-centric definition!
* A matrix is a grid representing many vectors (yes, it could be other things to, but this is good enough).
* We can also call these "Tensors..."  We'll be back to this.

### Vector Math (aka, linear algebra):
> What if I multiply our point, *(5,1)*, by 2?  If this coordinate defined the distance in X and Y from the origin of a point, multiplying by 2 would result in (10,2), *scaling* the value...  which is whyt we call them *scalars*

---

# Ok, that's enough Math...  What's the point

### There are very specific rules in this algebra
The only important rule that we need to be aware of is that all of our vectors or matrices *must* have the same number of dimensions, otherwise known as "columns" or "coordinates".

#### But Why, Dave?

**Because Linear Algebra is all about transforming things from one vector space to another!**


---

## With that done, let's create some vectors!
* Let's take our movie review and create a vector that has as many columns as we have words in our dictionary.
* If a word in our dictionary is present in the review, we flip that column "On" (set it to one)
* If a word doesn't appear in the review, it is left as zero

#### Here's some Python magic that does this for us:

In [ ]:
# We are very nearly ready to build a network.  First, recall that the "features" (data) must
# be "vectorized".  This means that we need to create a 1xN vector, where N is the number of
# features in the dataset.  Our main goal is to make all of the vectors the same dimensions.
# Matrix operations, which are at the heart of tensor calculus in machine learning.
#
# Since our machine learning approach also greatly prefers that all feature values lie between
# zero and one, we will take the opportunity to not only vectorize, but to one-hot encode the data!

# This is some Python magic that will take the array of reviews, which are sequences of word indices,
# and convert every row into a vector that has "WordsToWorkWith" dimensions.  If you've followed the
# directions so far, you are creating a 10,000 dimensional Tensor.  Wow!
# This function additionally puts a 1 at every position within a row that matches the index
# number of the words that are found in the review.
def vectorize_sequences(sequences, dimension=WordsToWorkWith):
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1.
    return results


x_train = vectorize_sequences(train_data)
x_test = vectorize_sequences(test_data)

# WHAT?!?  That's it?  Yes!  But what does it look like... Let's compare:
print(f"Here's what a row ({len(train_data[0])} elements long) from the dataset looks like to start with:\n\n{train_data[0]}\n")
print(f"Here's what that same row ({x_train[0].shape} dimensions) looks like when it has been vectorized:\n\n{x_train[0]}")

# Remember, this encoding is simply marking which words appear in the review
      

## A Bit More Data Manipulation and then Build the Network!

We can often do a better job of training if we give our network training data and labels and *validation data and labels*.  Validation data is data used during the training process to check how well the network performs on data that it has not been trained on, feeding that information back into the training process!

Let's pull out the validation data and then train our network!

In [ ]:
# First, we convert the labels from the format that they were in in the dataset to
# floating point numpy arrays
y_train = np.asarray(train_labels).astype('float32')
y_test = np.asarray(test_labels).astype('float32')

# Next, we split the training data at element 10,000 so that we have a validation set.
# Create a validation set from the training data
x_val = x_train[:10000]
partial_x_train = x_train[10000:]
y_val = y_train[:10000]
partial_y_train = y_train[10000:]

# Our neural network will have a number of sequential layers
model = models.Sequential()
# The first layer is a "Dense" or "Fully connected" layer, meaning that every neuron is connected to every other
# neuron in the previous and the next layers.  This layer will arbitrarily gave 16 neurons in it.
# Use the Rectified Linear function for activation.  Since this is
# the input layer, we must tell it what the shape of the input data is.
model.add(layers.Dense(16, activation='relu', input_shape=(WordsToWorkWith,)))
# Our model has one hidden layer, which is also a dense, or fully connected, layer.  It arbitrarily has 16 neurons.
model.add(layers.Dense(16, activation='relu'))
# Our final layer is a dense layer with only one neuron.  Why one neuron?  Because we are really looking for a
# binary answer; is this a positive or a negative review?  We apply the sigmoid function to this because the result
# is really a percentage between zero and one.
model.add(layers.Dense(1, activation='sigmoid'))

# Next we compile the model
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
# and then we fit the model to the data.  This is where the "Learning" happens!
training_history = model.fit(partial_x_train, partial_y_train, epochs=4, batch_size=512,validation_data=(x_val, y_val))


## Using the Neural Network
It might seem anticlimactic, but that's it!  You have now trained a neural network, which should have an accuracy greater than 90%, that is able to classify IMDB movie reviews as positive or negative!

How do we use it?  Let's hand our test data to the network, have it classify all 25,000 of those reviews, and see what we think of the output!

In [ ]:
# We've made this a separate step just so that you can appreciate what we have created.  While the training didn't
# take a very long time, some networks can take days, weeks, or more to train!  Still, using the network after
# training is typically very fast!
predictions = model.predict(x_test)
# That's it!  The model has been applied and the predictions have been made!  Let's look at the first five reviews 
# from the test data:

for i in range(0,5):
    goodBad = "Negative" if predictions[i] < 0.5  else "Positive"
    print("%s review: [%d%%]"%(goodBad, predictions[i]*100))
    print(decodeReview(test_data, i))
    print("------------------------------")

# Important Takeaways
There are some super important intuitions that you should have developed from this latest discussion:

* Look closely at the predictions, expecially the fourth prediction just above.  It is quite likely that your network has mis-classified this as a Positive review even though it is obviously negative.  *Machine learning algorithms **simulate** intelligence.  They do not actually **understand** anything.*
* This is really all just math.  A Neural Network, like the one above, is essentially just working out probabilities.  This is how all (most?) classification problems work, regardless of whether there are only two categories (binary) or many categories.
* Massaging the data into a format that can be used in a machine learning algorithm definitely takes some effort.
* When we structure the data that gets presented to the algorithm, all of the data must have the same number of dimensions.
    * These dimensions are just the number of terms in the vector (or ordered list of values).
